In [1]:
import nltk
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize
import re
import numpy as np
import math
import pandas as pd
from collections import Counter
import numpy as np
import pickle
import unicodedata
Stopwords = set(stopwords.words('english'))
ps=PorterStemmer()

In [2]:
def remove_special_characters(text):
    regex = re.compile('[^a-zA-Z\s]')
    text_returned = re.sub(regex,' ',text)
    return text_returned
def strip_accents(text):
    return ''.join(c for c in unicodedata.normalize('NFKD', text) if unicodedata.category(c) != 'Mn')

In [3]:
data=pd.read_csv('../English.csv')

In [4]:
with open('pickles/df.pkl','rb') as file:
    DF=pickle.load(file)
    file.close()
    
with open('pickles/file_idx.pkl','rb') as file:
    file_idx=pickle.load(file)
    file.close()
    
with open('pickles/doc_words.pkl','rb') as file:
    doc_words=pickle.load(file)
    file.close()
    
with open('pickles/doc_norm.pkl','rb') as file:
    doc_norm=pickle.load(file)
    file.close()

In [5]:
def tfidf(query,counter=10):
    query=strip_accents(query)
    text = remove_special_characters(query)
    text = re.sub(re.compile('\d'),'',text)
    words = word_tokenize(text)
    words = [word.lower() for word in words]
    words=[ps.stem(word) for word in words]
    words=[word for word in words if word not in Stopwords]
    words=[word for word in words if word in DF.keys()]

    q=[]
    q_norm=0
    for w in words:
        tf_idf=(words.count(w)*math.log(len(file_idx)/DF[w]))
        q.append(tf_idf)
        q_norm+=tf_idf**2
    q_norm=math.sqrt(q_norm)

    q=np.array(q)/q_norm

    score={}

    for i in range(len(file_idx)):
        doc_v=[]
        for w in words:
            tf_idf=(doc_words[i].count(w)*math.log(len(file_idx)/DF[w]))
            doc_v.append(tf_idf)
        doc_v=np.array(doc_v)/doc_norm[i]
        score[i]=np.dot(q,doc_v)

    score=sorted(score.items(),key=lambda x:x[1],reverse=True)
    verse_index=[]
    for i in score:
        if counter== 0:
            break
        verse_index.append(i[0])
        counter-=1
    return verse_index

In [6]:
verse_index=tfidf('bhagavad gita importance')
data.loc[verse_index]

,Chapter,Verse,Sloke,Commentary
587,18,68,य इदं परमं गुह्यं मद्भ‍क्तेष्वभिधास्यति ।भक्ति...,Generally it is advised that Bhagavad-gītā be ...
183,4,42,तस्मादज्ञानसम्भूतं हृत्स्थं ज्ञानासिनात्मनः ।छ...,The yoga system instructed in this chapter is ...
586,18,67,इदं ते नातपस्काय नाभक्ताय कदाचन ।न चाश‍ुश्रूषव...,Persons who have not undergone the austerities...
40,2,11,श्री भगवानुवाचअशोच्यनन्वशोचस्त्वं प्रज्ञावादां...,The Lord at once took the position of the teac...
303,8,28,वेदेषु यज्ञेषु तप:सु चैवदानेषु यत्पुण्यफलं प्र...,This verse is the summation of the Seventh and...
589,18,72,कच्च‍िदेतच्छ्रुतं पार्थ त्वयैकाग्रेण चेतसा ।कच...,The Lord was acting as the spiritual master of...
472,14,21,अर्जुन उवाचकैर्लिङ्गैस्त्रीन्गुणानेतानतीतो भवत...,"In this verse, Arjuna’s questions are very app..."
283,8,7,तस्मात्सर्वेषु कालेषु मामनुस्मर युध्य च ।मय्यर...,This instruction to Arjuna is very important f...
371,10,38,दण्डो दमयतामस्मि नीतिरस्मि जिगीषताम् ।मौनं चैव...,"There are many suppressing agents, of which th..."
347,10,14,सर्वमेतदृतं मन्ये यन्मां वदसि केशव ।न हि ते भग...,Arjuna herein confirms that persons of faithle...
